In [2]:
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.chrome.service import Service
import re

In [3]:
import csv
import pandas as pd
import re
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import spacy
from textblob import TextBlob
import streamlit as st
import plotly.express as px
import requests
import subprocess

nlp = spacy.load("en_core_web_sm")
#subprocess.run(["python", "-m", "spacy", "download", "en_core_web_sm"])
stop_words = set(stopwords.words('english'))


In [4]:
chrome_options = Options()
chrome_options.add_argument("--headless")
chrome_driver_path = "chromedriver.exe" 
service = Service(chrome_driver_path)
driver = webdriver.Chrome(service=service, options=chrome_options)

In [5]:
UrlProduct = 'https://www.amazon.com/EEYON-Baseball-Adjustable-Unstructured-Women-Grey/dp/B09ZQV36QJ/'
i = 1
UrlReview = UrlProduct.replace('dp','product-reviews') + '?pageNumber=' + str(i)
driver.get(UrlReview)
soup = BeautifulSoup(driver.page_source,'html.parser')
Product_name = soup.find('a',{'data-hook':"product-link"})
Product_name = str(Product_name.text)
Global_rating = soup.find('span',{'class':'a-color-secondary'})
# Global_rating = str(Global_rating.text).split(" ")[0]
rating = soup.select("#cm_cr-product_info > div > div.a-text-left.a-fixed-left-grid-col.reviewNumericalSummary.celwidget.a-col-left > div.a-row.a-spacing-small.averageStarRatingIconAndCount > div > div > div.a-fixed-left-grid-col.aok-align-center.a-col-right > div > span")
# rating = rating[0].text


In [6]:
driver.get(UrlProduct)
soup = BeautifulSoup(driver.page_source,'html.parser')
Image_of_product = soup.select('#landingImage')
Image_of_product = re.findall(r'src="https.*pg',str(Image_of_product[0]))
Image_of_product =  str(Image_of_product)
Image_of_product = Image_of_product[7:-2]

IndexError: list index out of range

In [17]:
def get_url_review_page(UrlProduct,i):
    UrlReview = UrlProduct.replace('dp','product-reviews') + '?pageNumber=' + str(i)
    return UrlReview
def get_rating(UrlProduct):
    driver.get(get_url_review_page(UrlProduct,1))
    soup = BeautifulSoup(driver.page_source,'html.parser')
    rating = soup.select("#cm_cr-product_info > div > div.a-text-left.a-fixed-left-grid-col.reviewNumericalSummary.celwidget.a-col-left > div.a-row.a-spacing-small.averageStarRatingIconAndCount > div > div > div.a-fixed-left-grid-col.aok-align-center.a-col-right > div > span")
    rating = str(rating)
    rating = str(rating)
    return rating[73:-17]
def get_product_name(UrlProduct):
    driver.get(get_url_review_page(UrlProduct,1))
    soup = BeautifulSoup(driver.page_source,'html.parser')
    Product_name = soup.find('a',{'data-hook':"product-link"})
    # Product_name = str(Product_name.string)
    Product_name = str(Product_name)
    pattern = r'>([^<]+)<'
    Product_name = re.findall(pattern, Product_name)
    return Product_name[0]
def get_global_rating(UrlProduct):
    driver.get(get_url_review_page(UrlProduct,1))
    soup = BeautifulSoup(driver.page_source,'html.parser')
    Global_rating = soup.find('span',{'class':'a-color-secondary'})
    Global_rating = str(Global_rating)
    return Global_rating[44:-7]
def get_image_url(UrlProduct):
    driver.get(UrlProduct)
    soup = BeautifulSoup(driver.page_source,'html.parser')
    Image_of_product = soup.select('#landingImage')
    Image_of_product = re.findall(r'src="https.*pg',str(Image_of_product[0]))
    Image_of_product =  str(Image_of_product)
    Image_of_product = Image_of_product[7:-2]
    return Image_of_product
def extract_reviews(UrlProduct):
    driver.get(get_url_review_page(UrlProduct,1))
    soup = BeautifulSoup(driver.page_source,'html.parser')
    review_spans = soup.find_all('span', class_='review-text-content')
    reviews = [review.get_text() for review in review_spans]
    return reviews
def extract_all_reviews(UrlProduct):
    all_reviews = []
    url_review = get_url_review_page(UrlProduct,1)
    numberOfpages = 10
    for i in range(1,numberOfpages+1):
        url_review = get_url_review_page(UrlProduct,1)
        reviews = extract_reviews(url_review)
        all_reviews.extend(reviews)
        # with open("file.txt", "a", encoding="utf-8") as file:
        #     for review in reviews:
        #         file.write(review + "\n")
        # print("Page", i)
    return all_reviews
def clean_comment(comment):
    comment = comment.lower()
    comment = re.sub(r'[^a-zA-Z\s]', '', comment)
    comment = re.sub(r'\d+', '', comment)
    # tokens = word_tokenize(comment)
    # tokens = [token for token in tokens if token not in stop_words]
    # cleaned_comment = ' '.join(tokens)
    return comment
def sentiment_analysis_textblob(text):
    doc = nlp(text)
    blob = TextBlob(doc.text)
    polarity = blob.sentiment.polarity
    if polarity > 0:
        sentiment_label = "Positive"
    elif polarity < 0:
        sentiment_label = "Negative"
    else:
        sentiment_label = "Neutral"
    return sentiment_label
def comment_dataFrame(UrlProduct):
    reviews = extract_all_reviews(UrlProduct)
    reviews_cleaned = [clean_comment(review.replace("\n", " ")) for review in reviews]
    df = pd.DataFrame(reviews_cleaned, columns=['comment'])
    df['sentiment'] = df['comment'].apply(sentiment_analysis_textblob)
    return df
def sentiemnt_by_comment(df):
    return df.groupby('sentiment').count()['comment'].reset_index()


In [ ]:
driver.get('https://www.amazon.com/kokib-Lightweight-Breathable-Athletic-Sneakers/dp/B08VW74CPR/')
soup = BeautifulSoup(driver.page_source,'html.parser')
x = soup.find('div',{'class':'a-section a-spacing-none aok-align-center aok-relative'})
x

In [ ]:
Product_name

'YEANIY Vintage Washed Cotton Plain Baseball Caps Adjustable Unstructured Soft Blank Casual Dad Ball Hats for Men Women-Grey'

In [ ]:
Global_rating = str(Global_rating)
Global_rating[44:-7]

'341 global ratings'

In [ ]:
rating
rating = str(rating)
print(rating[73:-17])

4.3


In [ ]:
Image_of_product

'https://m.media-amazon.com/images/I/81UEE11JmuL._AC_SX385_.jpg'

In [ ]:
Product_name = get_product_name(UrlProduct)
print(Product_name)
Product_name = str(Product_name)
pattern = r'>([^<]+)<'
Product_name = re.findall(pattern, Product_name)
Product_name

YEANIY Vintage Washed Cotton Plain Baseball Caps Adjustable Unstructured Soft Blank Casual Dad Ball Hats for Men Women-Grey


[]

In [13]:
Product_name[0]

IndexError: list index out of range

In [ ]:
get_image_url(UrlProduct)

'https://m.media-amazon.com/images/I/916XHpPQysL._AC_SX385_.jpg'

In [18]:
get_global_rating(UrlProduct)

''

In [41]:
def get_product_info(UrlProduct):
    driver.get(get_url_review_page(UrlProduct, 1))
    soup = BeautifulSoup(driver.page_source, 'html.parser')

    product_name_tag = soup.find('a', {'data-hook': "product-link"})
    product_name = re.findall(r'>([^<]+)<', str(product_name_tag))[0]

    global_rating_tag = soup.find('span', {'class': 'a-color-secondary'})
    global_rating = re.findall(r'>([^<]+)<', str(global_rating_tag))[0]


    image_tag = soup.select('#landingImage')
    image_url = re.findall(r'src="(https.*pg)', str(image_tag[0]))

    return product_name, global_rating, image_url


In [42]:
def get_product_info(UrlProduct):
    driver.get(get_url_review_page(UrlProduct, 1))
    soup = BeautifulSoup(driver.page_source, 'html.parser')

    product_name_tag = soup.find('a', {'data-hook': "product-link"})
    product_name = re.findall(r'>([^<]+)<', str(product_name_tag))[0]

    global_rating_tag = soup.find('span', {'class': 'a-color-secondary'})
    global_rating = re.findall(r'>([^<]+)<', str(global_rating_tag))[0]

    image_tag = soup.select('#landingImage')
    if image_tag:
        image_url = re.findall(r'src="(https.*pg)', str(image_tag[0]))
        if image_url:
            image_url = image_url[0]
        else:
            image_url = None
    else:
        image_url = None

    return product_name, global_rating, image_url


In [ ]:
get_product_info(UrlProduct)

IndexError: list index out of range

In [29]:
driver.get(get_url_review_page(UrlProduct, 1))
soup = BeautifulSoup(driver.page_source,'html.parser')
global_rating_tag = soup.find('span', {'class': 'a-color-secondary'})


In [28]:
get_url_review_page(UrlProduct,1)

'https://www.amazon.com/EEYON-Baseball-Adjustable-Unstructured-Women-Grey/product-reviews/B09ZQV36QJ/?pageNumber=1'

In [43]:
get_product_info(UrlProduct)

IndexError: list index out of range

In [18]:
df = comment_dataFrame(UrlProduct)


In [19]:
df.head()

,comment,sentiment
0,i have an average size head for a female but ...,Positive
1,just like i had hoped for super pricevalue,Positive
2,i needed a faded looking baseball cap for my ...,Positive
3,i liked both the look of the cap and the colo...,Positive
4,not much to say its a green cap and i like ho...,Positive


In [21]:
import pandas as pd
import plotly.express as px
from wordcloud import WordCloud
import matplotlib.pyplot as plt
positive_comments = df[df['sentiment'] == 'Positive']['comment']
positive_text = ' '.join(positive_comments)
wordcloud = WordCloud(width=800, height=400, background_color='white').generate(positive_text)
fig = px.imshow(wordcloud.to_array(), binary_string=True)
fig.update_layout(title='Word Cloud for Positive Comments')

Sujet 0:
the is fragile quality product with disappointed screen customer very
Sujet 1:
the is long lasts smartphone battery great new service responsive
